In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from aves.features.utils import normalize_rows, tfidf
from aves.models.datafusion import DataFusionModel
from gensim.utils import deaccent
from sklearn.metrics.pairwise import cosine_similarity

sns.set(
    style="whitegrid", context="paper", font_scale=0.8, font="Fira Sans Extra Condensed"
)
mpl.rcParams["figure.dpi"] = 120


In [ ]:
casen = pd.read_stata("../data/external/casen_2017/Casen 2017.dta", convert_categoricals=False)
casen.head()


In [ ]:
comunas = casen["comuna"].unique().astype(int)
comunas


In [ ]:
len(comunas)


In [ ]:
casen_2020 = pd.read_stata(
    "../data/external/casen_2020/Casen en Pandemia 2020 STATA.dta"
)
casen_2020.head()


In [ ]:
elecciones = pd.read_csv(
    "../data/external/presidenciales_2021/Servel_20211121_PRESIDENCIALES_CHILE.csv",
    sep=";",
)
elecciones.head()


In [ ]:
codes = pd.read_excel(
    "../data/external/CUT_2018_v04.xls"
)
codes


In [ ]:
codes["comuna_nombre"] = codes["Nombre Comuna"].str.upper().map(deaccent)
codes["comuna_nombre"]


In [ ]:
elecciones["votos_preliminar_string_strip"] = (
    elecciones["votos_preliminar_string"].str.strip().str.replace(r"^$", "0").astype(int)
)


In [ ]:
elecciones.groupby("candidato")["votos_preliminar_string_strip"].sum()


In [ ]:
elecciones["comuna_nombre"] = elecciones["comuna_nombre"].str.strip()
elecciones["comuna_nombre"]

In [ ]:
turnout = (
    elecciones.groupby(["comuna_nombre", "candidato"])["votos_preliminar_string_strip"]
    .sum()
    .unstack()
)
turnout.head()


In [ ]:
turnout.columns = list(map(lambda x: x.strip(), turnout.columns))
turnout.columns


In [ ]:
casen_comunas = casen.join(
    codes.set_index("Código Comuna 2018")["comuna_nombre"], on="comuna"
)
casen_comunas


In [ ]:
casen_2020["comuna_nombre"] = casen_2020["comuna"].str.upper().map(deaccent)


In [ ]:
poblacion_comunas = (
    casen_2020[casen_2020["edad"] >= 17].groupby("comuna_nombre")["expr"].sum()
)
poblacion_comunas


In [ ]:
poblacion_comunas.sum()


In [ ]:
candidatos = [
    "EDUARDO ARTES BRICHETTI",
    "GABRIEL BORIC FONT",
    "JOSE ANTONIO KAST RIST",
    "MARCO ENRIQUEZ-OMINAMI GUMUCIO",
    "FRANCO PARISI FERNANDEZ",
    "YASNA PROVOSTE CAMPILLAY",
    "SEBASTIAN SICHEL RAMIREZ",
]
columnas_votos = candidatos + ["Votos Blancos", "Votos Nulos"]
columnas_votos


In [ ]:
turnout = (
    turnout.reset_index()
    .assign(comuna_nombre=lambda x: x["comuna_nombre"].map(deaccent))
    .set_index("comuna_nombre")[columnas_votos]
)


In [ ]:
total_votos = turnout[columnas_votos].sum(axis=1)
total_votos


In [ ]:
abstencion = (poblacion_comunas - total_votos).rename("Abstención")
abstencion


In [ ]:
turnout = turnout.join(abstencion, how="left")[columnas_votos + ["Abstención"]]


In [ ]:
turnout.sum()


In [ ]:
stopwords = pd.read_csv(
    "../data/external/stopwords-es.txt",
    names=["word"],
)
stopwords.sample(10)


In [ ]:
from glob import glob

programas = sorted(
    glob(
        "../data/external/programas-presidenciales-2021/programas-primera-vuelta/*.txt"
    )
)
programas


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from aves.features.twokenize import tokenize

vectorizer = CountVectorizer(
    input="filename", tokenizer=tokenize, stop_words=list(stopwords["word"])
)
dtm = vectorizer.fit_transform(programas)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(sublinear_tf=True, norm="l2", smooth_idf=True)
weighted_dtm = tfidf.fit_transform(dtm)
weighted_dtm_df = pd.DataFrame(
    weighted_dtm.todense(), index=candidatos, columns=vectorizer.get_feature_names()
)
weighted_dtm_df = weighted_dtm_df[[c for c in weighted_dtm_df.columns if len(c) >= 3]]
weighted_dtm_df.loc["GABRIEL BORIC FONT"].sort_values(ascending=False).head(15)


In [ ]:
for c in candidatos:
    print(c)
    print(weighted_dtm_df.loc[c].sort_values(ascending=False).head(15).index)


In [ ]:
weighted_dtm_df_T = weighted_dtm_df.T

for c in turnout.columns:
    if not c in weighted_dtm_df_T.columns:
        weighted_dtm_df_T[c] = 0

weighted_dtm_df = weighted_dtm_df_T.T
weighted_dtm_df


## Matrices por comunas

In [ ]:
comunas_x_ingreso = (
    casen_comunas.groupby(["comuna_nombre", "dautr"])["expr"]
    .sum()
    .unstack(fill_value=0)
)
comunas_x_ingreso


In [ ]:
comunas_index = turnout.join(comunas_x_ingreso, how="inner").index
comunas_index


In [ ]:
comunas_x_ingreso = comunas_x_ingreso.loc[comunas_index]
comunas_x_ingreso

In [ ]:
comunas_x_drogas = (
    casen_comunas.groupby(["comuna_nombre", "v38b"])["expr"].sum().unstack(fill_value=0).loc[comunas_index]
)
comunas_x_drogas.columns = ["Nunca", "Pocas veces", "Muchas veces", "Siempre", "N/A"]
comunas_x_drogas


In [ ]:
casen_comunas["grupo_etareo"] = pd.cut(
    casen_comunas["edad"], bins=[-1, 17, 25, 35, 45, 55, 65, 75, 85, 95, 120]
)
comunas_x_edad = (
    casen_comunas.groupby(["comuna_nombre", "grupo_etareo"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_edad


In [ ]:
column_map = {1: "prepaid", 2: "contract", 3: "prepaid_and_contract", 4: "no", 9: "n/a"}
comunas_x_moviles = casen_comunas.pivot_table(
    index="comuna_nombre", columns="r22", values="expr", aggfunc="sum"
).fillna(0).loc[comunas_index]
comunas_x_moviles.columns = comunas_x_moviles.columns.map(lambda x: column_map[int(x)])
comunas_x_moviles


In [ ]:
comunas_x_trabajo = (
    casen_2020.groupby(["comuna_nombre", "o15"])["expr"].sum().unstack(fill_value=0).loc[comunas_index]
)
comunas_x_trabajo


In [ ]:
comunas_x_oficio = (
    casen_2020.groupby(["comuna_nombre", "oficio1_88"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_oficio


In [ ]:
comunas_x_origen = (
    casen_2020.groupby(["comuna_nombre", "r1b"])["expr"].sum().unstack(fill_value=0).loc[comunas_index]
)
comunas_x_origen


In [ ]:
comunas_x_migrantes = (
    casen_2020.groupby(["comuna_nombre", "r1b_p_cod"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_migrantes


In [ ]:
comunas_x_tipo_hogar = (
    casen_2020.groupby(["comuna_nombre", "tipohogar"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_tipo_hogar


In [ ]:
comunas_x_estado_civil = (
    casen_2020.groupby(["comuna_nombre", "ecivil"])["expr"]
    .sum()
    .unstack(fill_value=0)
    .drop("No sabe\\No responde", axis=1).loc[comunas_index]
)
comunas_x_estado_civil


In [ ]:
comunas_x_area = (
    casen_2020.groupby(["comuna_nombre", "area"])["expr"].sum().unstack(fill_value=0).loc[comunas_index]
)
comunas_x_area


In [ ]:
comunas_x_retiro = (
    casen_2020.groupby(["comuna_nombre", "o32b"])["expr"].sum().unstack(fill_value=0).loc[comunas_index]
)
comunas_x_retiro


In [ ]:
comunas_x_edad = (
    casen_2020[casen_2020["edad"] >= 17]
    .assign(
        grupo_etareo=lambda x: pd.cut(
            x["edad"], bins=[17, 25, 35, 45, 55, 65, 75, 85, 95, 120]
        )
    )
    .groupby(["comuna_nombre", "grupo_etareo"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_edad


In [ ]:
comunas_x_internet = (
    casen_comunas[casen_comunas["edad"] >= 18]
    .groupby(["comuna_nombre", "r21a"])["expr"]
    .sum()
    .unstack(fill_value=0).loc[comunas_index]
)
comunas_x_internet.columns = [
    "Usó redes sociales/Internet para informarse",
    "No usa redes sociales/Internet para informarse",
    "N/A",
]
comunas_x_internet


In [ ]:
delitos = pd.read_csv(
    "../data/external/delitos_cl_2016/Frec_CASOS_POLICIALES_ANUAL_2016.csv"
)
delitos


In [ ]:
comunas_x_delitos = (
    delitos[delitos["UN_ADMIN"] == "COMUNA"]
    .assign(
        comuna_nombre=lambda x: x["UNIDAD TERRITORIAL"]
        .str.replace("La Calera", "Calera")
        .str.upper()
        .map(deaccent)
    )
    .set_index("comuna_nombre")
    .drop(["ORDEN", "UN_ADMIN", "REGION", "UNIDAD TERRITORIAL"], axis=1).loc[comunas_index]
)

comunas_x_delitos


## Modelo de Fusión de Datos

In [ ]:
from aves.features.utils import tfidf as aves_tfidf

# dimensión (rank) de la representación latente de cada entidad
model_nodes = {
    "candidatos": 12,
    "comunas": 64,
    "grupo_etareo": 6,
    "programa": 24,
    "ingreso": 6,
    "celular": 2,
    "oficio": 8,
    "trabajo": 8,
    "retiro": 2,
    "origen": 2,
    "migrantes": 8,
    "hogar": 4,
    "narcotráfico": 3,
    "estado_civil": 4,
    "area": 2,
    "delincuencia": 4,
    "internet_inf": 2,
}

# relaciones entre entidades
# normalizamos todo
model_relations = {
    ("candidatos", "comunas"): [turnout.fillna(0).pipe(lambda x: aves_tfidf(x, norm='l2', smooth_idf=True)).T[comunas_index]],
    ("comunas", "grupo_etareo"): [
        comunas_x_edad.pipe(normalize_rows)
    ],
    ("comunas", "ingreso"): [comunas_x_ingreso.pipe(normalize_rows)],
    ("programa", "candidatos"): [weighted_dtm_df.T],
    ("comunas", "celular"): [comunas_x_moviles.pipe(normalize_rows)],
    ("comunas", "oficio"): [comunas_x_oficio.pipe(normalize_rows)],
    ("comunas", "trabajo"): [comunas_x_trabajo.pipe(normalize_rows)],
    ("comunas", "retiro"): [comunas_x_retiro.pipe(normalize_rows)],
    ("comunas", "origen"): [comunas_x_origen.pipe(normalize_rows)],
    ("comunas", "migrantes"): [
        comunas_x_migrantes.pipe(normalize_rows)
    ],
    ("comunas", "hogar"): [
        comunas_x_tipo_hogar.pipe(normalize_rows)
    ],
    ("comunas", "narcotráfico"): [
        comunas_x_drogas.pipe(normalize_rows)
    ],
    ("comunas", "estado_civil"): [
        comunas_x_estado_civil.pipe(normalize_rows)
    ],
    ("comunas", "area"): [comunas_x_area.pipe(normalize_rows)],
    ("comunas", "delincuencia"): [
        comunas_x_delitos.pipe(normalize_rows)
    ],
    ("comunas", "internet_inf"): [
        comunas_x_internet.pipe(normalize_rows)
    ],
}

model = DataFusionModel(nodes=model_nodes, relations=model_relations)
model.fit()


In [ ]:
import seaborn as sns

sns.heatmap(model.factor("candidatos"), annot=True)


In [ ]:
sns.heatmap(model.factor("candidatos").T.corr(), annot=True, center=0, cmap="PuOr_r")


In [ ]:
similarity = pd.DataFrame(
    cosine_similarity(model.factor("candidatos")),
    index=turnout.columns,
    columns=turnout.columns,
)
sns.heatmap(similarity, annot=True, cmap="Purples")


In [ ]:
model.relation_profiles("candidatos", "grupo_etareo")[0][1]

In [ ]:
from aves.features.utils import standardize_columns, standardize_rows

grid = sns.clustermap(
    model.relation_profiles("candidatos", "grupo_etareo")[0][1].pipe(
        standardize_columns
    ),
    col_cluster=False,
    figsize=(7, 4),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("Grupo Etáreo [años]")
grid.ax_heatmap.set_title(
    "Asociación entre Votación y Edad por Comunas (Fuentes: Servel, CASEN 2017, CASEN 2020)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
grid = sns.clustermap(
    model.relation_profiles("candidatos", "ingreso")[0][1].pipe(standardize_columns),
    col_cluster=False,
    figsize=(7, 4),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("Decil de Ingreso Autónomo Regional (mayor: más ingreso)")
grid.ax_heatmap.set_title(
    "Asociación entre Votación e Ingreso por Comunas (Fuentes: Servel, CASEN 2017, CASEN 2020)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
grid = sns.clustermap(
    model.relation_profiles("candidatos", "delincuencia")[0][1].T.pipe(
        standardize_rows
    ),
    col_cluster=True,
    figsize=(7, 5),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("")
grid.ax_heatmap.set_title(
    "Asociación entre Votación y Delitos de Connotación Social (Fuentes: Servel, CASEN 2017, CASEN 2020, SubSec. Prev. Delito)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
sns.heatmap(
    model.relation_profiles("candidatos", "narcotráfico")[0][1].pipe(standardize_rows),
    center=0,
)


In [ ]:
grid = sns.clustermap(
    model.relation_profiles("candidatos", "internet_inf")[0][1].T.pipe(
        standardize_rows
    ),
    col_cluster=True,
    figsize=(7, 3),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("")
grid.ax_heatmap.set_title(
    "Asociación entre Votación e Internet como fuente de información (Fuentes: Servel, CASEN 2017, CASEN 2020, SubSec. Prev. Delito)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
sns.heatmap(model.relation_profiles("candidatos", "celular")[0][1], center=0)


In [ ]:
grid = sns.clustermap(
    model.relation_profiles("candidatos", "trabajo")[0][1].T.pipe(standardize_rows),
    col_cluster=True,
    figsize=(7, 5),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("")
grid.ax_heatmap.set_title(
    "Asociación entre Votación y Tipo de Trabajo por Comuna (Fuentes: Servel, CASEN 2017, CASEN 2020)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
sns.heatmap(
    model.relation_profiles("candidatos", "area")[0][1].pipe(standardize_columns),
    center=0,
)


In [ ]:
sns.heatmap(model.relation_profiles("candidatos", "origen")[0][1], center=0)


In [ ]:
grid = sns.clustermap(
    model.relation_profiles("candidatos", "estado_civil")[0][1].T.pipe(
        standardize_rows
    ),
    col_cluster=False,
    figsize=(7, 4),
    annot=True,
    fmt=".2f",
    linewidth=0.5,
    dendrogram_ratio=[0.1, 0.0],
    method="ward",
    center=0,
    cmap="PuOr_r",
)

grid.ax_cbar.set_visible(False)
grid.ax_heatmap

grid.ax_heatmap.set_ylabel("")
grid.ax_heatmap.set_xlabel("")
grid.ax_heatmap.set_title(
    "Asociación entre Candidatos y Estado Civil por Comuna (Fuentes: Servel, CASEN 2017, CASEN 2020)",
    loc="left",
)

grid.fig.tight_layout()


In [ ]:
sns.clustermap(
    model.relation_profiles("candidatos", "migrantes")[0][1].pipe(standardize_columns),
    center=0,
    figsize=(18, 12),
    method="ward",
)


In [ ]:
sns.clustermap(
    model.relation_profiles("programa", "candidatos")[0][1]
    .pipe(standardize_rows)
    .pipe(lambda x: x[x.abs().max(axis=1) > 0.0002]),
    center=0,
    figsize=(12, 24),
    method="ward",
    col_cluster=False,
)
